In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, pipeline
from datasets import load_from_disk
from tqdm import tqdm
from vllm import LLM, SamplingParams
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from tqdm import tqdm
from datasets import Dataset

import os

/home/ubuntu/.conda/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-18 18:57:23,333	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def get_completion(prompts, model, tokenizer=None, max_tokens=4096, temperature=0, top_p=0.95):
    # 创建采样参数。temperature 控制生成文本的多样性，top_p 控制核心采样的概率
    sampling_params = SamplingParams(temperature=temperature, top_p=top_p,  max_tokens=max_tokens)
    # 初始化 vLLM 推理引擎
    llm = LLM(model=model, tokenizer=tokenizer, trust_remote_code=True, tensor_parallel_size=1, max_seq_len_to_capture=8192)
    outputs = llm.generate(prompts, sampling_params, use_tqdm=True)
    return outputs

In [3]:
# ins_prompt = load_from_disk("/home/ubuntu/test_disk/ins1_prompt")
ins_prompt = load_from_disk("/home/ubuntu/test_disk/ins2_prompt")

ins_prompt = ins_prompt["prompt"][:100]
# print(prompts[0])
model = "/home/ubuntu/test_disk/qwen7b"
ins_outputs = get_completion(ins_prompt, model, temperature=0.9, top_p=0.9)

INFO 12-18 18:57:43 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-18 18:57:43 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='/home/ubuntu/test_disk/qwen7b', speculative_config=None, tokenizer='/home/ubuntu/test_disk/qwen7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/ubuntu/test_di

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.16it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.20it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.14it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.13it/s]



INFO 12-18 18:57:47 model_runner.py:1077] Loading model weights took 14.2487 GB
INFO 12-18 18:57:50 worker.py:232] Memory profiling results: total_gpu_memory=23.65GiB initial_memory_usage=14.75GiB peak_torch_memory=18.60GiB memory_usage_post_profile=14.77GiB non_torch_memory=0.51GiB kv_cache_size=2.17GiB gpu_memory_utilization=0.90
INFO 12-18 18:57:50 gpu_executor.py:113] # GPU blocks: 2544, # CPU blocks: 4681
INFO 12-18 18:57:50 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 1.24x
INFO 12-18 18:57:53 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-18 18:57:53 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INF

Processed prompts:   0%|          | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 12-18 18:58:09 scheduler.py:1481] Sequence group 81 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 100/100 [00:06<00:00, 15.77it/s, est. speed input: 7584.47 toks/s, output: 583.66 toks/s]


In [5]:
ins_output_list = []
for i, output in enumerate(tqdm(ins_outputs)):
    ins_output_list.append(output.outputs[0].text)
ins_output_list[:5]

100%|██████████| 100/100 [00:00<00:00, 72779.87it/s]


['{\n  "quality": 9,\n  "difficulty": 8,\n  "relevance_to_medicine": 5,\n  "mention_specific_details": "False"\n}',
 '{\n  "quality": 7,\n  "difficulty": 8,\n  "relevance_to_medicine": 3,\n  "mention_specific_details": "False"\n}',
 '{\n  "quality": 9,\n  "difficulty": 6,\n  "relevance_to_medicine": 5,\n  "mention_specific_details": "False"\n}',
 '{\n  "quality": 9,\n  "difficulty": 8,\n  "relevance_to_medicine": 6,\n  "mention_specific_details": "False"\n}',
 '{\n  "quality": 9,\n  "difficulty": 7,\n  "relevance_to_medicine": 2,\n  "mention_specific_details": "False"\n}']

In [7]:
# 初始化存储成功解析的 JSON 数据和成功序号的列表
data = []
successful_indexes = []

# 使用列表推导解析 JSON，跳过有错误的项
for idx, item in enumerate(ins_output_list):
    try:
        # 尝试解析 JSON
        parsed_item = json.loads(item)
        data.append(parsed_item)
        successful_indexes.append(idx)  # 保存成功解析的序号
    except json.JSONDecodeError:
        # 如果解析失败，打印错误信息并跳过当前项
        print(f"第 {idx} 项解析失败")
        continue
# 使用 datasets 库创建数据集
ins_score_set = Dataset.from_list(data)

# 将 "ids" 列表添加到数据集中
ins_score_set = ins_score_set.add_column("ids", successful_indexes)

# 输出数据集内容
print(ins_score_set)

Dataset({
    features: ['quality', 'difficulty', 'relevance_to_medicine', 'mention_specific_details', 'ids'],
    num_rows: 100
})


In [8]:
ins_score_set[0]

{'quality': 9,
 'difficulty': 8,
 'relevance_to_medicine': 5,
 'mention_specific_details': 'False',
 'ids': 0}

In [7]:
# ins_score_set.save_to_disk("/home/ubuntu/test_disk/ins1_score")
ins_score_set.save_to_disk("/home/ubuntu/test_disk/ins2_score")

Saving the dataset (1/1 shards): 100%|██████████| 100/100 [00:00<00:00, 15254.79 examples/s]
